In [1]:

from osdatahub import FeaturesAPI, Extent, NGD
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os
import numpy as np
from convertbng.util import convert_bng, convert_lonlat

key = os.environ.get('OS_API_KEY')
crs = "EPSG:27700"

In [2]:
def OSparam_feature(u,v,rad,product,key,clip):
    extent = Extent.from_radius((u,v), rad, "EPSG:27700")
    features = FeaturesAPI(key, product, extent)
    results = features.query(limit=500)
    if len(results['features']) == 0:
        out = 0
    TA_gdf = gpd.GeoDataFrame.from_features(results['features'])

    if clip == True:
        patch = Point(u,v).buffer(rad)
        d = {'col1': ['name1'], 'geometry': [patch]}
        patch_df = gpd.GeoDataFrame(d, crs="EPSG:27700")
        TA_gdf = TA_gdf.clip(patch)
    try:
        gd = (TA_gdf['Theme'] == 'Buildings') & (~np.isnan(TA_gdf['RelH2']))
        out = np.average(TA_gdf['RelH2'][gd], weights=TA_gdf['Shape_Area'][gd])
    except TypeError:
        out = 0
    return out, TA_gdf



In [24]:
X = 180700.
Y = 44777.
radius = 100
clip = True
product = 'topographic_area'

In [25]:
def building_height_radius(X, Y, radius,product,key, clip):
    area, TA = OSparam_feature(X, Y, radius, product,key, clip)
    TA = TA.set_crs(27700)
    TA_ll = TA.to_crs(4326)
    lon, lat = convert_lonlat(X,Y)
    map = TA_ll.explore('RelH2') ## colour by height
    folium.Marker([lat[0], lon[0]],popup=[lat[0],lon[0]]).add_to(map)
    return map
    

In [26]:
building_height_radius(X, Y, radius,product,key, clip)

In [51]:
map = folium.Map(location=[51, 1], zoom_start=12)

# sim_geo = gpd.GeoSeries(TA["geometry"]).simplify(tolerance=0.001)
# geo_j = TA.to_json()

TA = TA.to_crs(epsg=4326)

for _, r in TA.iterrows():
    #without simplifying the representation of each borough, the map might not be displayed
    #sim_geo = gpd.GeoSeries(r['geometry'])
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['RelH2']).add_to(geo_j)
    geo_j.add_to(map)
map

ValueError: Cannot transform naive geometries.  Please set a crs on the object first.